In [15]:
! pip install -r requirements.txt
! playwright install

  Using cached crawl4ai-0.7.4-py3-none-any.whl.metadata (39 kB)
  Using cached langchain_text_splitters-0.3.11-py3-none-any.whl.metadata (1.8 kB)
  Using cached jupyter-1.0.0-py2.py3-none-any.whl.metadata (995 bytes)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached sentence_transformers-3.4.1-py3-none-any.whl.metadata (10 kB)
  Using cached scikit_learn-1.6.1-cp311-cp311-macosx_10_9_x86_64.whl.metadata (31 kB)
  Using cached rank_bm25-0.2.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached rouge_score-0.1.2-py3-none-any.whl
  Using cached bert_score-0.3.13-py3-none-any.whl.metadata (15 kB)
  Using cached pathlib-1.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached aiofiles-25.1.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached aiosqlite-0.21.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached lxml-5.4.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata (3.5 kB)
  Using cached playwright-1.55.0-py3-none-macosx_11_0_universal2.whl.metadata (3.5 kB)
  Using cache

In [16]:
import json
import os
from typing import List, Tuple

import dspy
import httpx
from dotenv import load_dotenv
from tqdm import tqdm

from src.dataclass import RetrievedDocument, LiteratureSearchAgentResponse, LiteratureSearchAgentRequest
from src.encoder import Encoder
from src.literature_search import LiteratureSearchAgent
from src.lm import init_lm, LanguageModelProviderConfig, LanguageModelProvider, LiteLLMServerConfig
from src.retriever_agent.serper_rm import SerperRM
from src.rag import RagAgent
from src.dataclass import RagResponse, RagRequest

load_dotenv()

True

In [17]:
## Action item 1. 
load_dotenv()

test_lm_config = LanguageModelProviderConfig(
      provider=LanguageModelProvider.LANGUAGE_MODEL_PROVIDER_LITELLM_SERVER,
      model_name="gpt-4.1-mini",
      temperature=0.0,
      max_tokens=10,
      litellm_server_config=LiteLLMServerConfig(api_key=os.getenv("LITELLM_API_KEY"), api_base=os.getenv("LITELLM_API_BASE"))
  )
test_lm = init_lm(test_lm_config)
print(test_lm("say 'Hello!' as is")[0]) # Expect to see "Hello!" or something similar

Hello!


In [18]:
import requests
import json

# Configure your API key
API_KEY = "sk-RG-FxDXC3TZW5eEQsUPdhg"
BASE_URL = "https://cs224v-litellm.genie.stanford.edu"

# Make the request
response = requests.post(
    f"{BASE_URL}/chat/completions",
    headers={
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    },
    json={
        "model": "gpt-4.1",
        "messages": [
            {"role": "user", "content": "Say hi in one word"}
        ],
        "max_tokens": 5
    }
)

# Check if request was successful
if response.status_code == 200:
    result = response.json()
    print("Success!")
    print(f"Response: {result['choices'][0]['message']['content']}")
    print(f"Tokens used: {result.get('usage', {}).get('total_tokens', 'N/A')}")
else:
    print(f"Error {response.status_code}: {response.text}")

Success!
Response: Hello!
Tokens used: 15


In [19]:
encoder = Encoder(model_name="text-embedding-3-small", **{"api_key": os.getenv("LITELLM_API_KEY"), "api_base": os.getenv("LITELLM_API_BASE")})
embedding = await encoder.aencode("hello")
assert len(embedding[0]) == 1536
print(f"✅ encoder is working")

✅ encoder is working


In [30]:
import importlib
importlib.reload(src.memory_layer)
import src.memory_layer as memory_layer
import src.lm
memory_layer.run_tests()

Starting Memory System Tests...

Adding test memories...
analysis {'keywords': ['neural networks', 'layers', 'neurons', 'process', 'information'], 'context': 'This content explains the basic structure and function of neural networks, highlighting that they consist of layers of neurons which process information, aimed at readers interested in understanding foundational concepts in artificial intelligence and machine learning.', 'tags': ['artificial intelligence', 'machine learning', 'neural networks', 'educational', 'technology']}
prompt_memory 
                                You are an AI memory evolution agent responsible for managing and evolving a knowledge base.
                                Analyze the the new memory note according to keywords and context, also with their several nearest neighbors memory.
                                Make decisions about its evolution.  

                                The new memory context:
                                This content exp

AttributeError: 'types.SimpleNamespace' object has no attribute 'category'

In [ ]:
# First, let's install the missing packages with compatible versions
import subprocess
import sys

def install_packages():
    """Install packages with compatible versions"""
    try:
        # Install numpy first with a compatible version
        subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy>=1.26.0"])
        
        # Install sentence-transformers 
        subprocess.check_call([sys.executable, "-m", "pip", "install", "sentence-transformers>=2.0.0"])
        
        print("✅ Successfully installed compatible packages")
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to install packages: {e}")
        return False

# Try to install packages
if install_packages():
    # Clear module cache
    modules_to_clear = [name for name in sys.modules.keys() if name.startswith('src.')]
    for module in modules_to_clear:
        if module in sys.modules:
            del sys.modules[module]

    print("Cleared cached modules:", modules_to_clear)
    print("Kernel ready for fresh imports!")
    
    # Now try to import and test
    try:
        import src.memory_layer as memory_layer
        import src.lm
        memory_layer.run_tests()
    except Exception as e:
        print(f"Error running memory layer tests: {e}")
        print("Let's check the current NumPy version:")
        import numpy
        print(f"NumPy version: {numpy.__version__}")
else:
    print("Please manually install the required packages and try again.")

In [ ]:
# Fix PyTorch-NumPy Compatibility
import subprocess
import sys

def fix_pytorch_numpy_compatibility():
    """Install compatible PyTorch and NumPy versions"""
    try:
        print("Installing compatible PyTorch and NumPy versions...")
        
        # Install compatible versions
        # PyTorch 2.4.0 is compatible with NumPy 1.24.3-2.1.x
        subprocess.check_call([
            sys.executable, "-m", "pip", "install", 
            "torch==2.4.0", 
            "numpy>=1.24.3,<2.1.0",
            "--force-reinstall"
        ])
        
        print("✅ Successfully installed compatible versions")
        print("Please restart the kernel to apply changes")
        return True
        
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to install compatible versions: {e}")
        return False

# Run the fix
fix_pytorch_numpy_compatibility()

In [ ]:
# Alternative Fix: Use CPU-only PyTorch with compatible NumPy
import subprocess
import sys

def fix_pytorch_alternative():
    """Install compatible PyTorch (latest) with compatible NumPy"""
    try:
        print("Installing latest PyTorch with compatible NumPy...")
        
        # Uninstall current torch first, then install latest with CPU-only
        subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "torch", "-y"])
        
        # Install latest PyTorch CPU version with NumPy 1.26
        subprocess.check_call([
            sys.executable, "-m", "pip", "install", 
            "torch", 
            "numpy==1.26.4"  # This version is known to work well with PyTorch
        ])
        
        print("✅ Successfully installed compatible versions")
        return True
        
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to install: {e}")
        return False

# Run the alternative fix
if fix_pytorch_alternative():
    print("\n🔄 Please restart the kernel to apply changes, then run the diagnostic again")
else:
    print("\nManual fix needed - see instructions below")